# Testing the Models

This notebook contains the steps for testing how well the Greek-Latin Identification and Author Reconciliation models perform on data from HathiTrust.

Running this notebook requires that the following files are in the same directory:

- authors_db.csv
- hathi2.csv
- utilities.py
- works_db.csv

These files are available in the Git repository https://github.com/DigitalLatin/dll_catalog_reconciliation.

## Load the Necessary Modules

This notebook uses the following modules:

- `csv`: for processing the output of the model
- `codecarbon`: to track energy usage
- `os`: for interacting with the operating system
- `pandas`: for opening and working with the CSV files
- `time`: for timing operations
- `torch`: for machine learning operations
- `transformers`: for loading the models and using them
- `utilities`: for some local helper functions. Note: this is a local file, not a library available through repos like condaforge or pypl.

Note that a HuggingFace access token is required. For information, see <https://huggingface.co/docs/hub/security-tokens>.

It is also likely that you will need to restart and run the following cell more than once.

In [21]:
!pip install codecarbon
!pip install --upgrade numpy datasets

[codecarbon INFO @ 14:17:32] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:17:32] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:17:32] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 14:17:32] 0.000219 kWh of electricity used since the beginning.


[codecarbon INFO @ 14:17:47] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:17:47] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:17:47] Energy consumed for All CPU : 0.000354 kWh
[codecarbon INFO @ 14:17:47] 0.000438 kWh of electricity used since the beginning.


In [22]:
import csv
from codecarbon import EmissionsTracker
import os
import pandas as pd
import time
import torch
import torch.nn.functional as F
from transformers.models.auto.tokenization_auto import AutoTokenizer
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers.models.distilbert import DistilBertForSequenceClassification, DistilBertTokenizerFast
import utilities as utilities

## Select the Device for Processing

The following block chooses "CUDA" if an NVIDIA GPU is available; "MPS" if an Apple Silicon GPU is available; or "CPU" in the absence of a GPU.

In [23]:
# General device selection for Colab (CUDA), Mac (MPS), or CPU fallback
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using device: CUDA (GPU)")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device: MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print("Using device: CPU")

Using device: CPU


## Load the DLL Catalog Data

The authority and work records from the DLL Catalog are loaded from CSV files and converted into Python dictionaries. These will be used as lookup tables to translate the outputs from the Author Reconciliation model into more comprehensible information for humans (e.g., "Julius Caesar", not "A4644").

In [24]:
# Read in the authors data
authors = pd.read_csv('./authors_db.csv',encoding='utf-8',quotechar='"')
# Read in the works data
works = pd.read_csv('./works_db.csv',encoding='utf-8',quotechar='"')

# Change the names of the columns to be lower case without spaces or punctuation
authors = authors.rename(columns={'Variant':'variant_name','Authorized Name':'authorized_name','DLL Identifier (Author)':'dll_id_author'})
works = works.rename(columns={'Title':'title','DLL Identifier (Work)':'dll_id_work','DLL Identifier (Author)': 'dll_id_author'})

# Prepare the lookup dictionaries of variant author names and titles
variant_to_authorized, title_to_work = utilities.prepare_dicts(authors,works)

[codecarbon INFO @ 14:18:02] Energy consumed for RAM : 0.000125 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:18:02] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:18:02] Energy consumed for All CPU : 0.000532 kWh
[codecarbon INFO @ 14:18:02] 0.000657 kWh of electricity used since the beginning.


## Load the Models

### Greek-Latin Identification Model

In [25]:
# Path to the Greek-Latin Identification Model
greek_latin_model_repo = "sjhuskey/distilbert_multilingual_cased_greek_latin_classifier"

# Load the model and tokenizer from Hugging Face Hub
greek_latin_model = DistilBertForSequenceClassification.from_pretrained(greek_latin_model_repo)
# Move the model to the appropriate device
greek_latin_model.to(device)
# Load the tokenizer
greek_latin_tokenizer = DistilBertTokenizerFast.from_pretrained(greek_latin_model_repo)

print("Model and tokenizer loaded successfully!")
# Verify label mappings
label2id = greek_latin_model.config.label2id
id2label = greek_latin_model.config.id2label

print("Label-to-ID Mapping:", label2id)
print("ID-to-Label Mapping:", id2label)

Model and tokenizer loaded successfully!
Label-to-ID Mapping: {'Greek': 0, 'Latin': 1}
ID-to-Label Mapping: {0: 'Greek', 1: 'Latin'}


### Author Reconciliation Model

In [26]:
# Path to Author Reconciliation Model
author_matching_repo = 'sjhuskey/distilbert_multilingual_cased_latin_author_identifier'

# Load the model for author reconciliation
author_matching = AutoModelForSequenceClassification.from_pretrained(author_matching_repo)
# Move the model to the appropriate device
author_matching.to(device)
# Load the tokenizer for author reconciliation
author_matching_tokenizer = AutoTokenizer.from_pretrained(author_matching_repo)

print("Author Reconciliation Model loaded successfully!")

# Verify label mappings
label2id = author_matching.config.label2id
id2label = author_matching.config.id2label

print("Label-to-ID Mapping:", label2id)
print("ID-to-Label Mapping:", id2label)

Author Reconciliation Model loaded successfully!
Label-to-ID Mapping: {'A1868': 1960, 'A1870': 2246, 'A2181': 2618, 'A2491': 14, 'A2492': 2290, 'A2493': 2215, 'A2494': 1752, 'A2495': 1554, 'A2508': 2051, 'A2755': 152, 'A2868': 365, 'A2870': 2274, 'A2871': 2455, 'A2872': 2372, 'A2873': 2917, 'A2874': 2558, 'A2875': 2821, 'A2876': 2690, 'A2877': 2536, 'A2878': 2419, 'A2879': 1455, 'A2880': 3074, 'A2881': 2952, 'A2882': 2626, 'A2883': 91, 'A2884': 2846, 'A2885': 1127, 'A2886': 2669, 'A2887': 1630, 'A2888': 733, 'A2889': 2798, 'A2890': 2396, 'A2891': 465, 'A2892': 2437, 'A2893': 3078, 'A2894': 2814, 'A2895': 2550, 'A2896': 1013, 'A2897': 498, 'A2898': 3053, 'A2901': 2435, 'A2902': 2147, 'A2903': 1443, 'A2904': 2325, 'A2905': 2570, 'A2906': 2606, 'A2907': 1235, 'A2908': 1622, 'A2909': 1236, 'A2910': 765, 'A2911': 330, 'A2912': 3049, 'A2913': 2385, 'A2914': 3077, 'A2915': 764, 'A2916': 2894, 'A2917': 2795, 'A2918': 2788, 'A2919': 2465, 'A2920': 127, 'A2921': 1530, 'A2922': 2941, 'A2923': 146

## Metadata Processing Functions

The following functions are necessary for providing comprehensible output from the models' inferences.

### Greek-Latin Identification

- `classify_author_language`: Tokenizes authors' names from the incoming metadata records and passes them to the Greek-Latin Identification model. Returns the predicted label ("Greek" or "Latin") and the confidence score for the prediction.
- `classify_and_split_by_language`: Returns three Pandas dataframes—`classified_df`, with all results from the Greek-Latin Identification model; `greek_df`, with results labeled "Greek"; `latin_df`, with results labeled "Latin". The latter will be the input for the Author Reconciliation model.

Note that these functions process one input at a time. There is potential for speeding up the process by batching the inputs.

In [27]:
# Language Classification Functions
def classify_author_language(input_author):
    """Classify the author's language as Greek or Latin using the fine-tuned model."""
    if not isinstance(input_author, str):
        return "Unknown", 0.0

    # Tokenize and encode the input author name
    inputs = greek_latin_tokenizer(input_author, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = greek_latin_model(**inputs)
    logits = outputs.logits.detach().cpu()

    # Apply softmax to logits to get probabilities
    probabilities = F.softmax(logits, dim=-1).numpy()
    predicted_class = probabilities.argmax(axis=-1)[0]
    confidence = probabilities.max()  # Highest probability

    # Use the id2label mapping for label names
    predicted_label = greek_latin_model.config.id2label[predicted_class]
    print(f"Language Classification: {input_author} -> {predicted_label} (Confidence: {confidence:.4f})")

    return predicted_label, confidence

def classify_and_split_by_language(processed_df):
    """Classify authors as Greek or Latin, add language info, and split the dataframe."""
    language_results = []

    for _, row in processed_df.iterrows():
        input_author = row["author"]

        # Perform language classification
        author_language, language_confidence = classify_author_language(input_author)

        # Add the results to a new row while preserving existing data
        updated_row = {
            **row,  # Include all original columns
            "language": author_language,
            "language_confidence": language_confidence,
        }
        language_results.append(updated_row)

    # Create a dataframe with the updated rows
    classified_df = pd.DataFrame(language_results)

    # Split the dataframe into Greek and Latin subsets
    greek_df = classified_df[classified_df["language"] == "Greek"].reset_index(drop=True)
    latin_df = classified_df[classified_df["language"] == "Latin"].reset_index(drop=True)

    return classified_df, greek_df, latin_df

### Author Reconciliation Functions

- `distilbert_author_match`: Tokenizes authors' names and passes them to the Author Reconciliation model. Returns the predicted label ("DLL ID") and the confidence score for the prediction.
- `process_metadata`: Manages input and output for the Author Reconciliation model. Returns a dataframe with the results of the inference run.

Note that these functions process one input at a time. There is potential for speeding up the process by batching the inputs.

In [28]:
# Author Reconciliation Functions
def distilbert_author_match(input_author):
    if not isinstance(input_author, str):
        return None, 0.0

    inputs = author_matching_tokenizer(input_author, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = author_matching(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    confidence = torch.softmax(logits, dim=-1).max().item()

    predicted_author_id = author_matching.config.id2label[predicted_class]
    for variant, author_info in variant_to_authorized.items():
        if author_info["author_id"] == predicted_author_id:
            return author_info, confidence
    return None, 0.0

def process_metadata(input_df):
    """Process input dataframe and match metadata."""
    results = []

    for _, row in input_df.iterrows():
        input_author_original = row["author"]
        input_author_normalized = utilities.normalize_author_name(input_author_original)
        print(f'Processing: {input_author_original}')

        # Author Matching
        distilbert_author, confidence = distilbert_author_match(input_author_original)
        if distilbert_author is None:
            print(f"No match found for author: {input_author_original}")
            continue
        # Collect Results
        results.append({
            "author": input_author_original,
            "normalized_author": input_author_normalized,
            "distilbert_author": distilbert_author,
            "confidence": confidence
        })
        print(f"Matched author: {distilbert_author}")

    return pd.DataFrame(results)

## Greek-Latin Identification Inferencing

The next few cells have to do with running the Greek-Latin Identification model on the data downloaded from the HathiTrust Digital Library. The original data file is in `../data/1908698974-1722799169.txt`. The cleaned and deduplicated version is at `../data/hathi2.csv`.

### Load the Data

In [29]:
# Load preprocessed, deduplicated hathi2.csv
input_df = pd.read_csv('./hathi2.csv', encoding='utf-8', quotechar='"')
# Further clean the data by filling missing values with "Unknown"
input_df = utilities.clean_input(input_df)

### Start the emissions tracker for the Greek-Latin Identification model

The `EmissionsTracker` from `codecarbon` will track the energy used by the Greek-Latin Identification model during inferencing.

In [30]:
# Create a logs directory
if not os.path.exists("./logs"):
    os.makedirs("./logs")

# Set up codecarbon's EmissionsTracker for the Greek-Latin Identification Model
tracker = EmissionsTracker(
    output_dir="./logs",
    output_file="greek_latin_identification_inference_emissions_log.csv"
)

tracker.start()
# Set up codecarbon's EmissionsTracker for the Greek-Latin Identification Model
tracker = EmissionsTracker(
    output_dir="./logs",
    output_file="greek_latin_identification_inference_emissions_log.csv"
)
tracker.start()

[codecarbon WARNING @ 14:18:04] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:18:04] [setup] RAM Tracking...
[codecarbon INFO @ 14:18:04] [setup] CPU Tracking...
[codecarbon WARNING @ 14:18:06] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 14:18:06] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 14:18:06] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 14:18:06] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 14:18:06] [setup] GPU Tracking...
[codecarbon INFO @ 14:18:06] No GPU found.
[codecarbon INFO @ 14:18:06] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

### Set up a Timer for the Greek-Latin Identification Inferencing

In [31]:
start_time = time.time()

### Begin Inferencing with the Greek-Latin Identification Model

In [32]:
# Signal the start of the metadata processing
print('Classifying authors as Greek or Latin …')
# Use the classify_and_split_by_language() function to run the Greek-Latin classification model
classified_df, greek_df, latin_df = classify_and_split_by_language(input_df)
# Signal the end of the Greek-Latin classification
print("Done with classification.")

Classifying authors as Greek or Latin …
Language Classification: Du Creux, François, 1596?-1666. -> Latin (Confidence: 1.0000)
Language Classification: Meyer, Ernst H. F. 1791-1858. -> Latin (Confidence: 1.0000)
Language Classification: Laet, Joannes de, 1593-1649. -> Latin (Confidence: 1.0000)
Language Classification: Caesar, Julius -> Latin (Confidence: 1.0000)
Language Classification: Unknown -> Latin (Confidence: 1.0000)
Language Classification: Drexel, Jeremias, 1581-1638, -> Latin (Confidence: 1.0000)
Language Classification: Kircher, Athanasius, 1602-1680 -> Latin (Confidence: 1.0000)
Language Classification: Hincmar, Archbishop of Reims, approximately 806-882 -> Latin (Confidence: 1.0000)
Language Classification: Acosta, José de, 1540-1600, -> Latin (Confidence: 1.0000)
Language Classification: Lessius, Leonardus, 1554-1623 -> Latin (Confidence: 1.0000)
Language Classification: Riccioli, Giovanni Battista, 1598-1671, -> Latin (Confidence: 1.0000)
Language Classification: Guazz

[codecarbon INFO @ 14:18:17] Energy consumed for RAM : 0.000167 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:18:17] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:18:17] Energy consumed for All CPU : 0.000709 kWh
[codecarbon INFO @ 14:18:17] 0.000875 kWh of electricity used since the beginning.


Language Classification: Juvenal. -> Latin (Confidence: 1.0000)
Language Classification: Kircher, Athanasius, 1602-1680 -> Latin (Confidence: 1.0000)
Language Classification: Claudianus, Claudius -> Latin (Confidence: 1.0000)
Language Classification: Knabenbauer, Joseph, 1839-1911. -> Latin (Confidence: 1.0000)
Language Classification: Suárez, Francisco, 1548-1617. -> Latin (Confidence: 1.0000)
Language Classification: Florus, Lucius Annaeus. -> Latin (Confidence: 1.0000)
Language Classification: Bellarmino, Roberto Francesco Romolo, Saint, 1542-1621. -> Latin (Confidence: 1.0000)
Language Classification: Suarez, Francisco, 1548-1617, -> Latin (Confidence: 1.0000)
Language Classification: Propertius, Sextus. -> Latin (Confidence: 1.0000)
Language Classification: Noceti, Carlo, 1694-1759. -> Latin (Confidence: 1.0000)
Language Classification: Maffei, Giovanni Pietro, 1536?-1603. -> Latin (Confidence: 1.0000)
Language Classification: Thomas, Aquinas, Saint, 1225?-1274. -> Latin (Confide

[codecarbon INFO @ 14:18:21] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:18:21] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:18:21] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 14:18:21] 0.000219 kWh of electricity used since the beginning.


Language Classification: Vater, Johann Severin, 1771-1826 -> Latin (Confidence: 1.0000)
Language Classification: John Chrysostom, Saint, -407 -> Greek (Confidence: 0.9995)
Language Classification: Clement, of Alexandria, Saint, approximately 150-approximately 215 -> Greek (Confidence: 0.9995)
Language Classification: Henrichsen, Rudolf Johann Frederik, 1800-1871. -> Latin (Confidence: 1.0000)
Language Classification: Herodotus -> Greek (Confidence: 0.9995)
Language Classification: Herodotus -> Greek (Confidence: 0.9995)
Language Classification: Lauer, Julius Franz, 1819-1850. -> Latin (Confidence: 1.0000)
Language Classification: Nemesius, Bp. of Emesa -> Greek (Confidence: 0.9995)
Language Classification: Unknown -> Latin (Confidence: 1.0000)
Language Classification: Pindar -> Greek (Confidence: 0.9995)
Language Classification: Groke, Th. Wilhelm. -> Latin (Confidence: 1.0000)
Language Classification: Theocritus. -> Greek (Confidence: 0.9995)
Language Classification: Xenophon -> Greek

...

Language Classification: Virgil. -> Latin (Confidence: 1.0000)
Language Classification: Īshō y̐abh III, Patriarch of the Nestorians, d. 657 or 8. -> Latin (Confidence: 0.9978)
Language Classification: Bābai, the elder, 569 (ca.)-628. -> Latin (Confidence: 0.9998)
Language Classification: Archivio di Stato (Siena, Italy) -> Latin (Confidence: 1.0000)
Language Classification: Widukind, of Corvey, approximately 925-approximately 973 -> Latin (Confidence: 1.0000)
Language Classification: Orsi, Roberto. -> Latin (Confidence: 1.0000)
Language Classification: Denari, Odofredo, -1265 -> Latin (Confidence: 1.0000)
Language Classification: Unknown -> Latin (Confidence: 1.0000)
Language Classification: Petronius Arbiter. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Seneca, Lucius Annaeus, approximately 4 B.C.-65 A.D. -> Latin (Co

[codecarbon INFO @ 14:33:08] Energy consumed for RAM : 0.002498 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:33:08] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:33:08] Energy consumed for All CPU : 0.010619 kWh
[codecarbon INFO @ 14:33:08] 0.013117 kWh of electricity used since the beginning.


Language Classification: Schrijver, Pieter, 1576-1660. -> Latin (Confidence: 1.0000)
Language Classification: Fabricius, Johann Albert, 1668-1736, -> Latin (Confidence: 1.0000)
Language Classification: Boeckh, August, 1785-1867. -> Latin (Confidence: 1.0000)
Language Classification: Livy. -> Latin (Confidence: 1.0000)
Language Classification: Tibullus. -> Latin (Confidence: 1.0000)
Language Classification: Virgil. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Unknown -> Latin (Confidence: 1.0000)
Language Classification: Juvenal. -> Latin (Confidence: 1.0000)
Language Classification: Pliny, the Younger. -> Latin (Confidence: 1.0000)
Language Classification: Quintilian. -> Latin (Confidence: 1.0000)
Language Classification: Tacitus, Cornelius. -> Latin (Confidence: 1.0000)
Language Classification: Tacitus, Cornelius. -> Latin (Confidence: 1.0000)
Language Classification: Valerius Maximus. -> Latin (C

[codecarbon INFO @ 14:33:17] Energy consumed for RAM : 0.002665 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:33:17] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:33:17] Energy consumed for All CPU : 0.011329 kWh
[codecarbon INFO @ 14:33:17] 0.013994 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:33:17] 0.006596 g.CO2eq/s mean an estimation of 208.02347355793444 kg.CO2eq/year


Language Classification: Petronius Arbiter. -> Latin (Confidence: 1.0000)
Language Classification: Lucretius Carus, Titus. -> Latin (Confidence: 1.0000)
Language Classification: Dubrovnik (Republic) -> Latin (Confidence: 1.0000)
Language Classification: Schuster, Ildefonso, 1880-1954. -> Latin (Confidence: 1.0000)
Language Classification: Dietrich, von Nieheim, ca. 1340-1418. -> Latin (Confidence: 1.0000)
Language Classification: Suárez, Francisco, 1548-1617. -> Latin (Confidence: 1.0000)
Language Classification: Vries, Hugo de, 1848-1935. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Curtius Rufus, Quintus. -> Latin (Confidence: 1.0000)
Language Classification: Cicero, Marcus Tullius. -> Latin (Confidence: 1.0000)
Language Classification: Pan

### Stop the Timer and Emissions Tracker for Greek-Latin Identification

In [33]:
# End the timer
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")
# Stop the emissions tracker after training is complete
emissions = tracker.stop()
print(f"Estimated CO2 emissions for training: {emissions} kg")

[codecarbon INFO @ 14:33:21] Energy consumed for RAM : 0.002533 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:33:21] Delta energy consumed for CPU with constant : 0.000148 kWh, power : 42.5 W
[codecarbon INFO @ 14:33:21] Energy consumed for All CPU : 0.010767 kWh
[codecarbon INFO @ 14:33:21] 0.013299 kWh of electricity used since the beginning.


Execution time: 912.64 seconds
Estimated CO2 emissions for training: 0.006019498022424989 kg


### Results of the Greek-Latin Identification Process

In [34]:
print(f"Number of rows in the Greek dataframe: {len(greek_df)}")
print(f"Number of rows in the Latin dataframe: {len(latin_df)}")
print(f"The classification step removed {len(classified_df)-len(latin_df)} records.")

Number of rows in the Greek dataframe: 1432
Number of rows in the Latin dataframe: 13491
The classification step removed 1432 records.


In [36]:
# Save the results to CSV files
classified_df.to_csv("./classified_metadata.csv", index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
greek_df.to_csv("./greek_authors.csv", index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
latin_df.to_csv("./latin_authors.csv", index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)

In [37]:
# Print unique authors in the Greek dataframe for verification
for author in greek_df['author'].unique():
    print(author)

Arrian.
Euclid,
Herodotus.
Celsus, Aulus Cornelius.
Anacreon
John Chrysostom, Saint, -407
Clement, of Alexandria, Saint, approximately 150-approximately 215
Herodotus
Nemesius, Bp. of Emesa
Pindar
Theocritus.
Xenophon
Frontinus, Sextus Julius
Catherine, of Alexandria, Saint.
Aristophanes.
Index librorum prohibitorum.
Homer
Epiphanius, Saint, Bishop of Constantia in Cyprus, approximately 310-403.
Euripides
Philo, of Alexandria
Plato.
Plato
Plotinus
Demosthenes
Plutarch.
Homer.
Theophrastus.
Thucydides
Epictetus
Origen
Celsus, Aulus Cornelius
Epiphanius, Saint, Bishop of Constantia in Cyprus, approximately 310-403
Aristotle
Archimedes.
Dioscorides Pedanius, of Anazarbos
Nicomachus, of Gerasa.
Proclus, approximately 410-485
Xenophon, of Ephesus
Gregory, of Nyssa, Saint, approximately 335-approximately 394.
Theocritus
Sextus, Empiricus
Philostratus, the Athenian, active 2nd century-3rd century
Longus
Stobaeus
Apollonius, of Athens.
Arrian
Frontinus, Sextus Julius.
Philo, of Byzantium
Justi

Those results look pretty solid.

## Author Reconciliation Inferencing

The remaining cells take the `latin_df` output from the Greek-Latin Identification model and submit it to the Author Reconciliation model for inferencing.

### Set up the emissions tracker for the Author Identification model

This, too, uses the `EmissionsTracker` from `codecarbon`.

In [38]:
# Set up CodeCarbon's EmissionsTracker
tracker = EmissionsTracker(
    output_dir="./logs",
    output_file="author_reconciliation_inference_emissions_log.csv"
)
tracker.start()

[codecarbon WARNING @ 14:36:47] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:36:47] [setup] RAM Tracking...
[codecarbon INFO @ 14:36:47] [setup] CPU Tracking...
[codecarbon INFO @ 14:36:47] Energy consumed for RAM : 0.003248 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:36:47] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:36:47] Energy consumed for All CPU : 0.013808 kWh
[codecarbon INFO @ 14:36:47] 0.017056 kWh of electricity used since the beginning.
[codecarbon WARNING @ 14:36:48] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 14:36:48] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 14:36:48] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20G

### Start a timer for Author Identification Inferencing

In [39]:
start_time = time.time()

### Begin Inferencing with the Author Identification Model

The following cell uses the `process_metadata()` function defined earlier to send data to the model for inferencing.

In [40]:
output_df = process_metadata(latin_df)
print("Done with processing authors.")

Processing: Du Creux, François, 1596?-1666.
Matched author: {'authorized_name': 'cruz, luís da, 1543-1604', 'author_id': 'A4003'}
Processing: Meyer, Ernst H. F. 1791-1858.
Matched author: {'authorized_name': 'meyer, wilhelm, 1845-1917', 'author_id': 'A6252'}
Processing: Laet, Joannes de, 1593-1649.
Matched author: {'authorized_name': 'larroumet, gustave', 'author_id': 'A3227'}
Processing: Caesar, Julius
Matched author: {'authorized_name': 'caesar, julius', 'author_id': 'A4644'}
Processing: Unknown
Matched author: {'authorized_name': 'stephanus abbas 4. or 6th century', 'author_id': 'A3062'}
Processing: Drexel, Jeremias, 1581-1638,
Matched author: {'authorized_name': 'drepanius florus 5th century', 'author_id': 'A3094'}
Processing: Kircher, Athanasius, 1602-1680
Matched author: {'authorized_name': 'kircher, athanasius, 1602-1680', 'author_id': 'A4106'}
Processing: Hincmar, Archbishop of Reims, approximately 806-882
Matched author: {'authorized_name': 'hincmar, archbishop of reims', 'au

[codecarbon INFO @ 14:37:02] Energy consumed for RAM : 0.003290 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:37:02] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:37:02] Energy consumed for All CPU : 0.013985 kWh
[codecarbon INFO @ 14:37:02] 0.017274 kWh of electricity used since the beginning.


Matched author: {'authorized_name': 'du bellay, joachim, approximately 1522-1560', 'author_id': 'A4268'}
Processing: Suarez, Francisco, 1548-1617,
Matched author: {'authorized_name': 'suárez, francisco, 1548-1617', 'author_id': 'A4125'}
Processing: Propertius, Sextus.
Matched author: {'authorized_name': 'propertius, sextus', 'author_id': 'A4889'}
Processing: Noceti, Carlo, 1694-1759.
Matched author: {'authorized_name': 'nogara, bartolomeo, 1868-1954', 'author_id': 'A3301'}
Processing: Maffei, Giovanni Pietro, 1536?-1603.
Matched author: {'authorized_name': 'maffei, giovanni pietro, 1536?-1603', 'author_id': 'A3869'}
Processing: Thomas, Aquinas, Saint, 1225?-1274.
Matched author: {'authorized_name': 'thomas, aquinas, saint', 'author_id': 'A4997'}
Processing: Franchis, Francesco de.
Matched author: {'authorized_name': "franchi de' cavalieri, pius, 1869-1960", 'author_id': 'A3267'}
Processing: Pliny, the Elder
Matched author: {'authorized_name': 'pliny, the elder', 'author_id': 'A5537'}


[codecarbon INFO @ 14:37:03] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:37:03] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:37:03] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 14:37:03] 0.000219 kWh of electricity used since the beginning.


Matched author: {'authorized_name': 'curtius rufus, quintus', 'author_id': 'A4602'}
Processing: Florus, Lucius Annaeus
Matched author: {'authorized_name': 'florus, lucius annaeus', 'author_id': 'A5047'}
Processing: Unknown
Matched author: {'authorized_name': 'stephanus abbas 4. or 6th century', 'author_id': 'A3062'}
Processing: Burchard, Johann Friedrich Wilhelm,
Matched author: {'authorized_name': 'burchard, von reichenau', 'author_id': 'A4975'}
Processing: Vater, Johann Severin, 1771-1826
Matched author: {'authorized_name': 'franz, johannes, 1804-1851', 'author_id': 'A6096'}
Processing: Henrichsen, Rudolf Johann Frederik, 1800-1871.
Matched author: {'authorized_name': 'lichtenau, heinrich von', 'author_id': 'A5109'}
Processing: Lauer, Julius Franz, 1819-1850.
Matched author: {'authorized_name': 'schubart, johann heinrich christian, 1800-1885', 'author_id': 'A3197'}
Processing: Unknown
Matched author: {'authorized_name': 'stephanus abbas 4. or 6th century', 'author_id': 'A3062'}
Proce

...

Matched author: {'authorized_name': 'graevius, joannes georgius, 1632-1703', 'author_id': 'A6032'}
Processing: Storr, Rayner.
Matched author: {'authorized_name': 'stern, reinhard august heinrich, 1804-1863', 'author_id': 'A6097'}
Processing: Hoffmann, Emanuel, 1825-1900.
Matched author: {'authorized_name': 'hoffmann, christian gottfried, 1692-1735', 'author_id': 'A3377'}
Processing: Petronius Arbiter.
Matched author: {'authorized_name': 'petronius arbiter', 'author_id': 'A4886'}
Processing: Lucretius Carus, Titus.
Matched author: {'authorized_name': 'lucretius carus, titus', 'author_id': 'A5001'}
Processing: Dubrovnik (Republic)
Matched author: {'authorized_name': 'council of chalcedon (451 : chalcedon)', 'author_id': 'A5281'}
Processing: Schuster, Ildefonso, 1880-1954.
Matched author: {'authorized_name': 'ildephonsus, saint', 'author_id': 'A5513'}
Processing: Dietrich, von Nieheim, ca. 1340-1418.
Matched author: {'authorized_name': 'loër, dirk, approximately 1495-1554', 'author_id': 

[codecarbon INFO @ 14:51:33] Energy consumed for RAM : 0.002456 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:51:33] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:51:33] Energy consumed for All CPU : 0.010442 kWh
[codecarbon INFO @ 14:51:33] 0.012899 kWh of electricity used since the beginning.


Matched author: {'authorized_name': 'wycliffe, john, -1384', 'author_id': 'A4338'}
Processing: Buchanan, David, 1595?-1652?
Matched author: {'authorized_name': 'buchanan, george', 'author_id': 'A4730'}
Processing: Munkelif, Norway. (Benedictine Monastery)
Matched author: {'authorized_name': 'withof, johann hildebrand', 'author_id': 'A4772'}
Processing: Priscianus, Theodorus.
Matched author: {'authorized_name': 'priscianus, theodorus', 'author_id': 'A3493'}
Processing: Heydenreich, Wilhelm, 1875-
Matched author: {'authorized_name': 'heytesbury, william, active 1340', 'author_id': 'A4296'}
Processing: Sallust, 86-34 B.C.
Matched author: {'authorized_name': 'sallust', 'author_id': 'A5543'}
Processing: Tacitus, Cornelius.
Matched author: {'authorized_name': 'tacitus, cornelius', 'author_id': 'A4796'}
Processing: Tacitus, Cornelius.
Matched author: {'authorized_name': 'tacitus, cornelius', 'author_id': 'A4796'}
Processing: Tacitus, Cornelius.
Matched author: {'authorized_name': 'tacitus, co

[codecarbon INFO @ 14:51:37] Energy consumed for RAM : 0.005579 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:51:37] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 14:51:37] Energy consumed for All CPU : 0.023718 kWh
[codecarbon INFO @ 14:51:37] 0.029298 kWh of electricity used since the beginning.


Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Livy.
Matched author: {'authorized_name': 'livy', 'author_id': 'A4979'}
Processing: Lucan, 39-65.
Matched author: {'authorized_name': 'lucan', 'author_id': 'A5075'}
Processing: Petermann, August, 1818-
Matched author: {'authorized_name': 'peter, hermann, 1837-1914', 'author_id': 'A6260'}
Processing: Tacitus, Cornelius.
Matched author: {'authorized_name': 'tacitus, cornelius', 'author_id': 'A4796'}
Processing: Portus, Aemilius, 1550-1614 or 15.
Matched author: {'authorized_name': 'azo, portius, -ap

### End the Timer and the EmissionsTracker for Author Reconciliation

In [41]:
# End the timer
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")
# Stop the emissions tracker after training is complete
emissions = tracker.stop()
print(f"Estimated CO2 emissions for author reconciliation: {emissions} kg")

[codecarbon INFO @ 14:51:38] Energy consumed for RAM : 0.002468 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:51:38] Delta energy consumed for CPU with constant : 0.000051 kWh, power : 42.5 W
[codecarbon INFO @ 14:51:38] Energy consumed for All CPU : 0.010493 kWh
[codecarbon INFO @ 14:51:38] 0.012962 kWh of electricity used since the beginning.


Execution time: 885.60 seconds
Estimated CO2 emissions for author reconciliation: 0.005866802341367513 kg


### Display the First Ten Rows of the Output

In [42]:
display(output_df.head(10))

,author,normalized_author,distilbert_author,confidence
0,"Du Creux, François, 1596?-1666.",du creux francois 15961666,"{'authorized_name': 'cruz, luís da, 1543-1604...",0.467435
1,"Meyer, Ernst H. F. 1791-1858.",meyer ernst h f 17911858,"{'authorized_name': 'meyer, wilhelm, 1845-1917...",0.999939
2,"Laet, Joannes de, 1593-1649.",laet joannes de 15931649,"{'authorized_name': 'larroumet, gustave', 'aut...",0.494398
3,"Caesar, Julius",caesar julius,"{'authorized_name': 'caesar, julius', 'author_...",0.999999
4,Unknown,unknown,{'authorized_name': 'stephanus abbas 4. or 6th...,0.177456
5,"Drexel, Jeremias, 1581-1638,",drexel jeremias 15811638,{'authorized_name': 'drepanius florus 5th cent...,0.969138
6,"Kircher, Athanasius, 1602-1680",kircher athanasius 16021680,"{'authorized_name': 'kircher, athanasius, 1602...",0.999999
7,"Hincmar, Archbishop of Reims, approximately 80...",hincmar archbishop of reims approximately 806882,"{'authorized_name': 'hincmar, archbishop of re...",0.999999
8,"Acosta, José de, 1540-1600,",acosta jose de 15401600,"{'authorized_name': 'acosta, josé de, 1540-16...",1.000000
9,"Lessius, Leonardus, 1554-1623",lessius leonardus 15541623,"{'authorized_name': 'lessing, gotthold ephraim...",0.999962


### Write the results to a CSV

In [43]:
output_df.to_csv('./model_probabilistic_deterministic_comparison.csv',index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)